# Using `opto.trainer` algorithms for scaling up generative optimization

This tutorial walks you through the different algorithms that have been built on top of the generative optimizers in Trace.
The `minibatch` tutorial already showed one specific use-case: `MiniBatchAlgorithm` that takes an agent, dataset and opto optimizer as inputs and outputs an optimized agent. 
In fact, all of the algorithms in `opto.trainer` obey this basic input-output mapping; they all use the opto optimizers to propose candidate parameters, but utilize different search procedures on top of that to refine the optimized agent.

We will use the [HardMath dataset](https://huggingface.co/datasets/xuanfeiren/math_hard_gemini) in this tutorial to illustrate the various algorithms in `opto.trainer`.

In [4]:
%pip install trace-opt ipywidgets

Looking in indexes: https://pypi.netflix.net/simple
Note: you may need to restart the kernel to use updated packages.


The code below provides a way to specify your API_KEY for calling LLMs using LiteLLM as part of this tutorial notebook. Alternatively, provide the keys by setting environment variables or loading LiteLLM config files.

In [5]:
import os
import ipywidgets as widgets
from IPython.display import display

# Function to save the environment variable and API key
def save_env_variable(env_name, api_key):
    # Validate inputs
    if not env_name.strip():
        print("⚠️ Environment variable name cannot be empty.")
        return
    if not api_key.strip():
        print("⚠️ API key cannot be empty.")
        return
    
    # Store the API key as an environment variable
    os.environ[env_name] = api_key
    globals()[env_name] = api_key  # Set it as a global variable
    print(f"✅ API key has been set for environment variable: {env_name}")

# Create the input widgets
env_name_input = widgets.Text(
    value="OPENAI_API_KEY",  # Default value
    description="Env Name:",
    placeholder="Enter env variable name (e.g., MY_API_KEY)",
)

api_key_input = widgets.Password(
    description="API Key:",
    placeholder="Enter your API key",
)

# Create the button to submit the inputs
submit_button = widgets.Button(description="Set API Key")

# Display the widgets
display(env_name_input, api_key_input, submit_button)

# Callback function for the button click
def on_button_click(b):
    env_name = env_name_input.value
    api_key = api_key_input.value
    save_env_variable(env_name, api_key)

# Attach the callback to the button
submit_button.on_click(on_button_click)

Text(value='OPENAI_API_KEY', description='Env Name:', placeholder='Enter env variable name (e.g., MY_API_KEY)'…

Password(description='API Key:', placeholder='Enter your API key')

Button(description='Set API Key', style=ButtonStyle())

We load the dataset and define a `Guide` (i.e. LLM-as-Judge) that can provide feedback for answers to questions in the dataset.

In [6]:
import datasets
import numpy as np
from typing import Any, Tuple
from opto.trainer.guide import Guide, LLMJudge
from opto.utils.llm import LLM

# Set random seed
np.random.seed(42)

math_data = datasets.load_dataset('xuanfeiren/math_hard_gemini')
train_data = math_data['train'].select(
        range(10, 30)
    )
validate_data = train_data
test_data = math_data['test'].select(range(10))

# Format data for trainer
train_dataset = {'inputs': train_data['problem'], 'infos': train_data['solution']}
validate_dataset = {'inputs': validate_data['problem'], 'infos': validate_data['solution']}
test_dataset = {'inputs': test_data['problem'], 'infos': test_data['solution']}

# Log dataset sizes
print(f"Training samples: {len(train_dataset['inputs'])}")
print(f"Validation samples: {len(validate_dataset['inputs'])}")
print(f"Test samples: {len(test_dataset['inputs'])}")

# Use the built-in LLMJudge instead of creating a custom TeacherGuide
math_judge = LLMJudge(
    model="gpt-4o-mini",
    prompt_template=(
        "Carefully review the following three distinct sections:\n\n"
        "SECTION 1: The Math Problem\n"
        "----------------------------\n"
        "{query}\n"
        "----------------------------\n\n"
        "SECTION 2: The Student's Full Answer\n"
        "----------------------------\n"
        "{response}\n"
        "----------------------------\n\n"
        "SECTION 3: The Official Correct Answer\n"
        "----------------------------\n"
        "{reference}\n"
        "----------------------------\n\n"
        "INSTRUCTIONS FOR JUDGING:\n"
        "1. Your primary task is to compare the student's **final numerical result** (or final conclusion if no number is present) from SECTION 2 with the **Official Correct Answer** provided in SECTION 3.\n"
        "2. When evaluating SECTION 2 (Student's Full Answer), focus SOLELY on the **final answer part** of the student's response. Ignore all intermediate steps, reasoning, or explanations for the correctness check unless the problem specifically asks for reasoning as the final answer.\n"
        "3. Determine if the student's **final answer** is equivalent to the **Official Correct Answer**.\n\n"
        "RESPONSE FORMAT:\n"
        "- If the student's final answer (from SECTION 2) IS equivalent to the Official Correct Answer (from SECTION 3), respond ONLY with the exact phrase: '{correctness_template}'\n"
        "- If the student's final answer IS NOT equivalent, respond ONLY with '{incorrectness_template}' and provide specific and actionable feedback. The feedback should clearly explain the error in the student's final answer and guide them on how to arrive at the Official Correct Answer."
    ),
    system_prompt="You are an expert math teacher evaluating student answers."
)

Training samples: 20
Validation samples: 20
Test samples: 10


We define the `Learner` agent which is a student LLM with a trainable `system prompt` and a trainable `user prompt template`. Trace will use a generative optimizer to tune these prompts.

In [ ]:
from opto import trace, trainer
from opto.optimizers import OptoPrime
from opto.optimizers.utils import print_color
from opto.trainer.algorithms.basic_algorithms import MinibatchAlgorithm, BasicSearchAlgorithm
from opto.trainer.algorithms.beamsearch_algorithm import BeamsearchAlgorithm, BeamsearchHistoryAlgorithm
from opto.trainer.algorithms.UCBsearch import UCBSearchAlgorithm
from opto.features.predefined_agents import BasicLearner

# Create alias for backward compatibility in this tutorial
Learner = BasicLearner

We initialize all the components: the agent using the student LLM, the guide using the teacher LLM, and the optimizer using an LLM as a generative optimizer.

In [8]:
student_llm = LLM()
agent = Learner(llm=student_llm)

# Use the LLMJudge we created above for both training and validation
train_guide = math_judge
validate_guide = math_judge

optimizer = OptoPrime(agent.parameters())

from opto.trainer.loggers import DefaultLogger
class SimpleLogger(DefaultLogger):
    """Simplified logger that only shows important metrics."""
    
    def log(self, name: str, data: Any, step: int, **kwargs):
        """Log only specific metrics to reduce output clutter.
        
        Args:
            name: The name of the metric
            data: The metric value
            step: The current step
            **kwargs: Additional logging arguments
        """
        important_metrics = [
            'Average train score',
            'Average test score',
            'Validation score'
        ]
        
        if name in important_metrics or 'Parameter' in name:
            super().log(name, data, step, **kwargs)

logger = SimpleLogger()

import nest_asyncio
nest_asyncio.apply()
import asyncio

train_params = {
        "guide": train_guide,
        "train_dataset": train_dataset,
        "num_epochs": 1,
        "num_threads": 5,
        "batch_size": 5,
        "test_dataset": test_dataset,
        "validate_dataset": validate_dataset,
        "validate_guide": validate_guide,
        "eval_frequency": 2,
        "log_frequency": 2,
        #for Basic Search
        "num_proposals": 2,
        #for Beam Search
        "validation_dataset_size": 5,
        "beam_width": 3,
        "max_depth": 4,
        "max_history_size": 2,
        #for UCB Search
        "num_search_iterations": 3,
        "train_batch_size": 5,
        "evaluation_batch_size": 5,
        "max_buffer_size": 3,
        "ucb_exploration_factor": 1.0
    }

Finally, we will go through each of the algorithms in `opto.trainer`. Each algorithm will run the student model on the train dataset, gather feedback from the teacher model, present the resulting traced graph to the optimizer, and then perform specific post-processing throughout each training epoch.

In [9]:
algorithm = MinibatchAlgorithm(
            agent=agent,
            optimizer=optimizer,
            logger=logger,
            num_threads=train_params["num_threads"]
        )

async def wrapper():
    print("STARTING TRAINING MINIBATCH")
    metrics, final_score = algorithm.train(**train_params)
    print("FINISHED TRAINING MINIBATCH")
    print("Final score: ", final_score)

asyncio.run(wrapper())

STARTING TRAINING MINIBATCH


Evaluating agent (iteration 0): 100%|██████████| 10/10 [00:49<00:00,  4.94s/it]


[Step 0] Average test score: 0.1


Checking improvement (iteration 0): 100%|██████████| 5/5 [00:32<00:00,  6.41s/it]


Update rejected: Current score 0.0, New score 0.0


...

Update rejected: Current score 0.0, New score 0.0


Checking improvement (iteration 3): 100%|██████████| 5/5 [00:26<00:00,  5.23s/it]


Update accepted: Current score 0.2, New score 0.4


Evaluating agent (iteration 4): 100%|██████████| 10/10 [00:46<00:00,  4.62s/it]

[Step 4] Average test score: 0.4
Epoch: 0. Iteration: 4
[Step 4] Average train score: 0.05
[Step 4] Parameter: str:0: Adjusting calculations largely hinges on understanding exact component replacements or alternative setup checklists ('For each model', enrich sequential rotation-class opportunity collection), correcting calculations based on feedback: Producing combinations correctly and preferring direct statistically significant outcomes.
FINISHED TRAINING MINIBATCH
Final score:  0.4


In [10]:
algorithm = BasicSearchAlgorithm(
            agent=agent,
            optimizer=optimizer,
            logger=logger,
            num_threads=train_params["num_threads"]
        )

async def wrapper():
    print("STARTING TRAINING BASIC SEARCH")
    metrics, final_score = algorithm.train(**train_params)
    print("FINISHED TRAINING BASIC SEARCH")
    print("Final score: ", final_score)
    
asyncio.run(wrapper())

STARTING TRAINING BASIC SEARCH


Evaluating agent (iteration 0):   0%|          | 0/10 [00:00<?, ?it/s]

Evaluating agent (iteration 0): 100%|██████████| 10/10 [00:44<00:00,  4.42s/it]


[Step 0] Average test score: 0.2


Validating proposals: 100%|██████████| 20/20 [01:28<00:00,  4.45s/it]


...

[Step 2] Validation score: 0.25


Validating proposals: 100%|██████████| 20/20 [01:38<00:00,  4.95s/it]


[Step 3] Validation score: 0.25


Evaluating agent (iteration 4): 100%|██████████| 10/10 [00:53<00:00,  5.35s/it]

[Step 4] Average test score: 0.3
Epoch: 0. Iteration: 4
[Step 4] Average train score: 0.1
[Step 4] Parameter: str:0: Adjusting calculations largely hinges on understanding exact component replacements or alternative setup checklists ('For each model', enrich sequential rotation-class opportunity collection), correcting calculations based on feedback: Producing combinations correctly and preferring direct statistically significant outcomes.
FINISHED TRAINING BASIC SEARCH
Final score:  0.3


In [11]:
algorithm = BeamsearchAlgorithm(
            agent=agent,
            optimizer=optimizer,
            logger=logger,
            num_threads=train_params["num_threads"]
        )

async def wrapper():
    print("STARTING TRAINING BEAM SEARCH")
    metrics, final_score = algorithm.train(**train_params)
    print("FINISHED TRAINING BEAM SEARCH")

    if 'best_validation_scores' in metrics:
        print("\nBest validation scores at each depth:")
        for depth, score in enumerate(metrics['best_validation_scores']):
            print(f"  Depth {depth+1}: {score:.4f}")
            
    print("Final score: ", final_score)
    
asyncio.run(wrapper())

STARTING TRAINING BEAM SEARCH
Running BeamsearchAlgorithm with beam_width=3, max_depth=4
Using validation_dataset_size=5 for intermediate evaluations

===== Evaluating Initial Parameters =====


Evaluating initial parameters on test set:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluating initial parameters on test set: 100%|██████████| 10/10 [00:37<00:00,  3.71s/it]


Initial test score: 0.2000

===== Beam Search Depth 1/4 with 1 beams =====
Sampled validation minibatch of size 5 for depth 1
Processing beam 1/1


Generating 2 proposals for beam 1:  50%|█████     | 1/2 [00:09<00:09,  9.84s/it]

...

Candidate 2: Validation score: 0.0500


Validating candidate 3/3: 100%|██████████| 20/20 [01:56<00:00,  5.82s/it]


Candidate 3: Validation score: 0.0000
Selected top 1 beams with scores: ['0.0500']

===== Final Proposal Candidate Parameters =====
str:0: Calculating the correct outputs requires revisiting each mathematical problem's formulation. For ID [0], re-evaluate permutations while ensuring proper distinctions between ascending and descending orders to reach the correct count of 52. For ID [1], apply recursive relationships correctly accounting for decrement reductions, leading to 122 substitutions. In ID [2], follow Burnside's lemma with an exhaustive assessment of symmetries per official instructions to reach 336. ID [3] must accurately factor in population movement percentages, finally achieving 840. For ID [4], implement systematic optimal selections in modular arithmetic to achieve the full potential set selection of 905.


Evaluating best beam on test set: 100%|██████████| 10/10 [00:30<00:00,  3.08s/it]

BEST BEAM - Test score: 0.2000

===== Periodic Test Scores Summary =====
Depth 1: Test score = 0.2000
Depth 4: Test score = 0.3000
FINISHED TRAINING BEAM SEARCH

Best validation scores at each depth:
  Depth 1: 0.0000
  Depth 2: 0.2000
  Depth 3: 0.4000
  Depth 4: 0.2000
Final score:  0.2


In [12]:
algorithm = BeamsearchHistoryAlgorithm(
            agent=agent,
            optimizer=optimizer,
            logger=logger,
            num_threads=train_params["num_threads"]
        )

async def wrapper():
    print("STARTING TRAINING BEAM SEARCH w/ HISTORY")
    metrics, final_score = algorithm.train(**train_params)
    print("FINISHED TRAINING BEAM SEARCH w/ HISTORY")

    if 'best_validation_scores' in metrics:
        print("\nBest validation scores at each depth:")
        for depth, score in enumerate(metrics['best_validation_scores']):
            print(f"  Depth {depth+1}: {score:.4f}")
            
    print("Final score: ", final_score)
    
asyncio.run(wrapper())

STARTING TRAINING BEAM SEARCH w/ HISTORY
Running BeamsearchHistoryAlgorithm with beam_width=3, max_depth=4, max_history_size=2
Using validation_dataset_size=5 for intermediate evaluations

===== Evaluating Initial Parameters =====


Evaluating initial parameters on test set: 100%|██████████| 10/10 [00:32<00:00,  3.28s/it]


Initial test score: 0.2000

===== Beam Search Depth 1/4 with 1 beams =====
Sampled validation minibatch of size 5 for depth 1
Processing beam 1/1


Forward pass (beam 1, batch size: 5): 100%|██████████| 5/5 [00:23<00:00,  4.78s/it]
Generating 2 proposals for beam 1 (with history):   0%|          | 0/2 [00:00<?, ?it/s]

Generating 2 proposals for beam 1 (with history):  50%|█████     | 1/2 [00:13<00:13, 13.06s/it]

...

Candidate 2: Validation score: 0.1000


Validating candidate 3/3: 100%|██████████| 20/20 [01:42<00:00,  5.13s/it]


Candidate 3: Validation score: 0.1000
Selected top 1 beams with scores: ['0.1500']

===== Final Proposal Candidate Parameters =====


Evaluating best beam on test set: 100%|██████████| 10/10 [00:26<00:00,  2.63s/it]

BEST BEAM - Test score: 0.3000

===== Periodic Test Scores Summary =====
Depth 1: Test score = 0.2000
FINISHED TRAINING BEAM SEARCH w/ HISTORY

Best validation scores at each depth:
  Depth 1: 0.4000
  Depth 2: 0.2000
  Depth 3: 0.4000
  Depth 4: 0.4000
Final score:  0.3


In [13]:
algorithm = UCBSearchAlgorithm(
            agent=agent,
            optimizer=optimizer,
            logger=logger,
            num_threads=train_params["num_threads"],
            max_buffer_size=train_params["max_buffer_size"],
            ucb_exploration_factor=train_params["ucb_exploration_factor"]
        )

async def wrapper():
    print("STARTING TRAINING UCB SEARCH")
    metrics, final_score = algorithm.train(**train_params)
    print("FINISHED TRAINING UCB SEARCH")

    if 'best_candidate_scores' in metrics and metrics['best_candidate_scores']:
        print(f"  Best candidate scores over iterations: {len(metrics['best_candidate_scores'])} recorded")
        print(f"  Final best candidate score: {metrics['best_candidate_scores'][-1]:.4f}")
    if 'buffer_avg_score' in metrics and metrics['buffer_avg_score']:
        print(f"  Final buffer average score: {metrics['buffer_avg_score'][-1]:.4f}")
            
    print("Final score: ", final_score)
    
asyncio.run(wrapper())

STARTING TRAINING UCB SEARCH
Evaluating initial parameters using validation_dataset samples...


Evaluating candidate: 100%|██████████| 5/5 [00:28<00:00,  5.69s/it]


Initial candidate: Score 0.0000, Evals 5
Iter 1/3: 


Iter 1: Forward pass for action 'a' : 100%|██████████| 5/5 [00:18<00:00,  3.66s/it]


LLM response:
 {
"reasoning": "The #Instruction asks to improve the output in accordance with the #Feedback. This requires adjusting the values of #Variables. Each model call with different `message` and fixed `system_prompt` and `user_prompt_template` represents a different tasked model solving a specific problem. The #Feedback indicates the model outputs do not match the official correct answers. Analyzing each output and feedback suggestion, we need to adjust the `system_prompt` to guide the model to problem-specific strategies or reasoning that align with the official resolution approaches:\n\n1. ID [0]: The model's answer of 1512 should be 560. The `system_prompt` should better guide in terms of applying binomial coefficients and combinatorial reasoning specific to arranging subsequences.\n\n2. ID [1]: The computed 1750 fish should be 840. The `system_prompt` should strengthen understanding proportion calculations and assumptions about populations based on capture ratio.\n\n3. ID 

...

Iter 2: New candidate a_prime generated. Validation Score: 0.0000, Evals: 5
Iter 2: Added new candidate to buffer.
Log @ Iter 2: Best score in buffer: 0.2000, Buffer size: 3, Total samples: 25
Iter 3/3: 


Iter 3: Forward pass for action 'a' : 100%|██████████| 5/5 [00:23<00:00,  4.80s/it]


LLM response:
 {
"reasoning": "The instruction is to change the values of variables specified in #Variables to address the issues highlighted in the #Feedback. We have several calculations in #Outputs, each corresponding to a message that was submitted to the model for computation, and each feedback tells us the correct solution that should have been reached. For ID [0], the feedback indicates that the correct probability is 91/100, so the output needs to reflect this change. For ID [1], the correct number of typing orders is 704, suggesting an error in the approach. For ID [2], a miscalculation regarding the coach’s handshakes highlights the need for recalibrating based on a maximum arrangement configuration. For ID [3], a similar oversight occurs in the enumeration of card arrangements. Variable str0 combines different task interpretations leading each sub-problem; hence each element should properly guide the modeling or thinking for respective valid scenarios. Adjusting this might a

Evaluating candidate: 100%|██████████| 5/5 [00:33<00:00,  6.64s/it]

Iter 3: New candidate a_prime generated. Validation Score: 0.0000, Evals: 5
Iter 3: Buffer full. Evicted a candidate (UCB: 0.5963)
Iter 3: Added new candidate to buffer.
UCB search finished.
Final best candidate: Mean Score 0.2000, Evals 10
FINISHED TRAINING UCB SEARCH
  Best candidate scores over iterations: 3 recorded
  Final best candidate score: 0.2000
  Final buffer average score: 0.1000
Final score:  0.2


In [14]:
# Using the simplified trainer.train approach
from opto import trainer

# Create a fresh agent for simplified training
simple_agent = BasicLearner(
    system_prompt="You're a helpful agent answering math problems.",
    user_prompt_template="Solve the following math problem step-by-step: {message}",
    llm=LLM()
)

# Run MinibatchAlgorithm using trainer.train
print("STARTING SIMPLIFIED TRAINING")
metrics, final_score = trainer.train(
    model=simple_agent,
    train_dataset=train_dataset,
    algorithm='MinibatchAlgorithm',
    guide=math_judge,  # Use the same LLMJudge we created earlier
    # trainer kwargs
    num_epochs=1,
    batch_size=5,
    eval_frequency=2,
    test_dataset=test_dataset,
    num_threads=5,
    verbose='output',
)
print("FINISHED SIMPLIFIED TRAINING")
print(f"Final score: {final_score}")

STARTING SIMPLIFIED TRAINING


Evaluating agent (iteration 0): 100%|██████████| 10/10 [00:44<00:00,  4.41s/it]


[Step 0] Average test score: 0.1


Forward pass (batch size: 5): 100%|██████████| 5/5 [00:30<00:00,  6.04s/it]


LLM response:
 ```

<reasoning>
The instruction requires us to modify the variables in #Variables to improve the output based on the feedback. The feedback indicates issues with incorrect answers generated by the agent. Specifically, the feedback involves misunderstandings in solving math problems, where correct answers depend on understanding problem constraints and calculations.

The #Code uses `str188` as the `system_prompt` for the `BasicLearner.model` function calls. This prompt is essential in guiding the model's behavior and approach when addressing the queries. Currently, `str188` is set to: "You're a helpful agent answering math problems." From the feedback, it is evident that the answers provided do not meet expectations, so modifying this prompt could help guide the model to process tasks correctly.

By making `str188` more specific in terms of expectations and processing steps, we can direct the model to adopt a more meticulous approach to these mathematical problems. The c

...

LLM response:
 ```

<reasoning>
The `str188` variable, a system prompt saying "You're a helpful agent answering math problems," serves to define the role of the agent for providing mathematical solutions. The feedback indicates that the current outputs are incorrect answers to the math problems the agent tackled. Improving the output requires altering the prompt in a way that might better guide or inform the learning algorithm (or decision-making process) on how to approach mathematical problems with correct logic or consideration.

Considering #Feedback, errors occur because correct logic or systematic calculation wasn't applied. Modifying `str188` to insist on accuracy, methodical calculation, or reasoning logic could guide better answers.

Hence, updating `str188` to highlight logical reasoning would be sensible.
</reasoning>
<variable>
<name>str188</name>
<value>
You're a logical and methodical math problem-solving agent. Focus on accurate calculations and reasoning.
</value>
</var

Checking improvement (iteration 3): 100%|██████████| 5/5 [00:35<00:00,  7.07s/it]


Update accepted: Current score 0.0, New score 0.2


Evaluating agent (iteration 4): 100%|██████████| 10/10 [00:47<00:00,  4.78s/it]

[Step 4] Average test score: 0.2
Epoch: 0. Iteration: 4
[Step 4] Instantaneous train score: 0.0
[Step 4] Average train score: 0.05
[Step 4] Parameter: str:188: You're a logical and methodical math problem-solving agent. Focus on accurate calculations and reasoning.
FINISHED SIMPLIFIED TRAINING
Final score: 0.2


## Simplified Training with `trainer.train()`

Instead of manually setting up the algorithm, optimizer, guide, and logger, you can use the simplified `trainer.train()` function that handles all the setup for you. This is the recommended approach for most use cases.

The `trainer.train()` function:
- Automatically selects the appropriate optimizer based on your model type
- Uses sensible defaults for guide and logger
- Provides a clean, unified interface for all training algorithms
- Reduces boilerplate code significantly

Let's see some examples:

In [15]:
# Example: Using trainer.train with different algorithms
print("="*50)
print("TRAINING WITH BASIC SEARCH ALGORITHM")
print("="*50)

# Create another fresh agent
basic_search_agent = BasicLearner(
    system_prompt="You're a math tutor providing step-by-step solutions.",
    user_prompt_template="Problem: {message}\n\nSolution:",
    llm=LLM()
)

metrics, final_score = trainer.train(
    model=basic_search_agent,
    train_dataset=train_dataset,
    algorithm='BasicSearchAlgorithm',
    guide=math_judge,
    num_epochs=1,
    batch_size=3,
    num_proposals=2,
    test_dataset=test_dataset,
    validate_dataset=validate_dataset,
    validate_guide=math_judge,
    num_threads=3,
)
print(f"Basic Search final score: {final_score}")

print("="*50)
print("TRAINING WITH BEAM SEARCH ALGORITHM")
print("="*50)

# Create another fresh agent for beam search
beam_search_agent = BasicLearner(
    system_prompt="You are an expert mathematician.",
    user_prompt_template="Mathematical Problem: {message}\n\nDetailed Solution:",
    llm=LLM()
)

metrics, final_score = trainer.train(
    model=beam_search_agent,
    train_dataset=train_dataset,
    algorithm='BeamsearchAlgorithm',
    guide=math_judge,
    num_epochs=1,
    batch_size=3,
    beam_width=2,
    max_depth=2,
    validation_dataset_size=5,
    test_dataset=test_dataset,
    validate_dataset=validate_dataset,
    validate_guide=math_judge,
    num_threads=3,
)
print(f"Beam Search final score: {final_score}")

TRAINING WITH BASIC SEARCH ALGORITHM


Evaluating agent (iteration 0): 100%|██████████| 10/10 [01:01<00:00,  6.13s/it]


[Step 0] Average test score: 0.3


Validating proposals: 100%|██████████| 20/20 [02:54<00:00,  8.73s/it]


[Step 0] Validation score: 0.25


...

Candidate 1: Validation score: 0.1000


Validating candidate 2/2: 100%|██████████| 20/20 [02:39<00:00,  7.97s/it]


Candidate 2: Validation score: 0.1000
Selected top 1 beams with scores: ['0.1000']
[Step 3] Final validation score: 0.1

===== Final Proposal Candidate Parameters =====
str:243: You are an expert in mathematical problem solving. Ensure thorough analysis of foundational conditions, detailed explanations of required steps, and exploration of alternative solution strategies, especially in probabilistic and sequence problems.


Evaluating best beam on test set: 100%|██████████| 10/10 [01:03<00:00,  6.32s/it]

BEST BEAM - Test score: 0.3000
[Step 3] Final test score: 0.3

===== Periodic Test Scores Summary =====
Depth 1: Test score = 0.3000
Beam Search final score: 0.3
